# Notes for next work

# Theory and Thought process

In [1]:
##Idea that faces touched is = edges vertex is on, seems to hold for all 3d platonic solids
    ## How to find every vertex that shares an edge, used here and in distance calculation
        ## Shares an edge if they share 2 or more faces
        ## How to find shared faces
## Just brute forcing through the faces listings in the inputMatrix works, but can be super slow
    ##For 600-face, around 200,000 actions, 2.5 to find certain vertex in face, scan 600 faces for 
    ##    first vertex, then that subset for second vertex (20), then 120 vertices
## Binary matrix probably easier, less searching
    ## Would just find each face that has the first vertex, then find each that has 2nd in that subset
        ## 600, then 20, by 120? Down below 100,000 actions
    ## Around 2 or 3 times faster
    
## Calculate distance of vertices to faces
    ## Find distance from a certain vertex to all vertices on a face, take minimum
    ## Needs to be upfront, not on call
    ## 1 unit away = shares an edge with a vertex on the face
        ## Share an edge if they share more than 1 face (ASK DR B IF THIS IS TRUE)**********************************
    ## 2 units away = shares an edge with a vertex that is 1 unit away from the face
    ## Need to use a path finding algorithm to find minimal distance from each vertex to each other vertex
        ## Ew, thats a ton of computation
    ## Store how?
        ## Could be a 2d array with an int value for number of edges away
            #rows, cols = (maxVert, maxVert)
            #print (rows, cols)
            #vertexListOfTouchingFaces = [[0 for i in range(cols)] for j in range(rows)]
            #print (vertexListOfTouchingFaces)
        ## Could be a 2d array where the cells are lists of vertices that are a certain distance away
        ## If just using vertex to vertex and doing multiple search per face instead of computations, should
        ##   probably use the first one
        ## Ex: d4
        ##   1 2 3 4
        ## 1 0 1 1 1
        ## 2 1 0 1 1
        ## 3 1 1 0 1
        ## 4 1 1 1 0
        
        ## Ex: d6
        ##   1 2 3 4 5 6 7 8
        ## 1 0 1 2 1 1 2 3 2
        ## 2 1 0 1 2 2 1 2 3 
        ## 3 2 1 0 1 3 2 1 2
        ## 4 1 2 1 0 2 3 2 1
        ## 5 1 2 3 2 0 1 2 1
        ## 6 2 1 2 3 1 0 1 2
        ## 7 3 2 1 2 2 1 0 1
        ## 8 2 3 2 1 1 2 1 0
        
        ## Ex: d8
        ##   1 2 3 4 5 6
        ## 1 0 1 1 1 1 2
        ## 2 1 0 1 2 1 1
        ## 3 1 1 0 1 2 1
        ## 4 1 2 1 0 1 1
        ## 5 1 1 2 1 0 1
        ## 6 2 1 1 1 1 0
        
        ## These matrices should always be symmetric on diagonal
        
        ## Filling these out would be like a tree, make the identity 0, fill out everything thats 1 away,
        ## Fill out everything 2 away, so on
        ## Finding stuff 2 away, find everything 1 away from vertices already 1 away, excluding what's already listed
        ## ............. 3 ..................... 1 .......................... 2 .....................................
        
        
        ## This means the creation of this matrix won't be row by row completely
        ## Easier to go row by row hitting everything 1 away, then row by row hitting everything 2 away
            ## Do this since for 2 away, need info of whats 1 away from every other vertex
            ## Read from notebook if can't visualize

# Imports

In [2]:
##IMPORTS
import numpy as np
from numpy import *
import copy
from collections import defaultdict
import heapq as heap

# Take in or create input

In [3]:

## FOR THE BIG 600 CELL ONE
import csv

with open("sixhundredcell_3_corrected.csv") as file_name:
    inputMatrix = np.loadtxt(file_name, dtype = 'int', delimiter=",")

    
## FOR SMALLER TESTING SOLIDS
#inputMatrix = array([[1,2,3,4],
#            [1,2,5,6],
#            [2,3,6,7],
#            [3,4,7,8],
#            [1,4,5,8],
#            [5,6,7,8]])

    


# Find total number of vertices

In [4]:
##use .shape for output

## Find number of vertices
inputMatrixWidth= len(inputMatrix[0])

maxVert = np.max(inputMatrix)
print(maxVert)

120


# Create master arrays for which faces have and haven't been covered

In [5]:
## Create Master Array of Uncovered Faces
## Will be removed from throughout process
masterUncoveredFaces = copy.deepcopy(inputMatrix)
print(masterUncoveredFaces)
print("")

##Create Master Array for covered faces, starts empty, fills to be inputMatrix

rows, cols = (1, inputMatrixWidth)
masterCoveredFaces = [[-1 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)


print(masterCoveredFaces)

[[  1  29  43  49]
 [  1  29  36  43]
 [  1  25  26  30]
 ...
 [ 89 106 111 116]
 [100 106 116 120]
 [ 89 100 106 116]]

[[-1, -1, -1, -1]]


# Creating the Dictionary of each face that touches each vertex

In [6]:
## Create list of faces touched by each vertex, as well as copy for strictly uncovered

## How to know how many faces each vertex will touch? **************************
    ## For platonic solid, equal for all, so find max faces for one, for all
    ## Just make it a dictionary, probably better

vertexDictOfTouchedFaces = {
    
}

for nextVertex in range(maxVert):
    vertexDictOfTouchedFaces[nextVertex+1] = []
    
## Fill in the dictionary
for currentFace in inputMatrix:
    for currentVertex in currentFace:
            vertexDictOfTouchedFaces[currentVertex].append(currentFace)
            
print(vertexDictOfTouchedFaces)


##Create the dictionary for each uncovered face, will be removed from throughout the process
vertexDictOfTouchingUncoveredFaces = copy.deepcopy(vertexDictOfTouchedFaces)

print(vertexDictOfTouchedFaces[1])
print(len(vertexDictOfTouchedFaces[1]))
    
##This is way easier
max_val = max(len(vertexDictOfTouchedFaces[i]) for i in vertexDictOfTouchedFaces)
print(max_val)


{1: [array([ 1, 29, 43, 49]), array([ 1, 29, 36, 43]), array([ 1, 25, 26, 30]), array([ 1, 25, 26, 29]), array([ 1, 25, 36, 37]), array([ 1, 33, 36, 37]), array([ 1, 33, 40, 44]), array([ 1, 33, 40, 43]), array([ 1, 40, 49, 50]), array([ 1, 26, 49, 50]), array([ 1, 30, 44, 50]), array([ 1, 30, 37, 44]), array([ 1, 25, 29, 36]), array([ 1, 33, 36, 43]), array([ 1, 40, 43, 49]), array([ 1, 26, 29, 49]), array([ 1, 25, 30, 37]), array([ 1, 33, 37, 44]), array([ 1, 40, 44, 50]), array([ 1, 26, 30, 50])], 2: [array([ 2, 29, 56, 62]), array([ 2, 29, 36, 62]), array([ 2, 25, 27, 32]), array([ 2, 25, 27, 29]), array([ 2, 25, 36, 39]), array([ 2, 34, 36, 39]), array([ 2, 34, 53, 64]), array([ 2, 34, 53, 62]), array([ 2, 53, 56, 58]), array([ 2, 27, 56, 58]), array([ 2, 32, 58, 64]), array([ 2, 32, 39, 64]), array([ 2, 25, 29, 36]), array([ 2, 34, 36, 62]), array([ 2, 53, 56, 62]), array([ 2, 27, 29, 56]), array([ 2, 25, 32, 39]), array([ 2, 34, 39, 64]), array([ 2, 53, 58, 64]), array([ 2, 27, 

# Create Dictionary Of Which Vertices a Single Vertex shares edges with

In [7]:
##Create something that stores every vertex, and each other vertex it shares an edge with
    ## shares edge if it shares 2 faces
    ##Look in vertexDictOfTouchedFaces, keep count of each time each vertex appears. If > 1 and not the original, edge
##What to store in, dict doesn't seem to be working well, hard with arrays as values
    ## Dict for now
    
vertexDictOfSharedEdges = {}

for nextVertex in range(maxVert):
    vertexDictOfSharedEdges[nextVertex+1] = []


rows, cols = (maxVert, maxVert)
vertexSharedFacesCount = [[0 for i in range(cols)] for j in range(rows)]
#print(len(vertexSharedFacesCount)*len(vertexSharedFacesCount[0]))



for currentVertex in vertexDictOfTouchedFaces:
    
    #print(currentVertex, vertexDictOfTouchedFaces[currentVertex])
    for currentFace in vertexDictOfTouchedFaces[currentVertex]:
        for vertexOnFace in currentFace:
            vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] += 1
            
            ## This if statement is the problem, assumes points only connect on 2 faces
                ## Or maybe not
            
            if vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] > 1 and vertexOnFace not in vertexDictOfSharedEdges[currentVertex]:
                    if vertexOnFace != currentVertex:
                        vertexDictOfSharedEdges[currentVertex].append(vertexOnFace)
    
    ##PUT THIS INSIDE LOOP ABOVE, SO TO ACCESS THE VALUE OF vertexOnFace
    ## Also, use number of faces each vertex is on to eliminate vertices pairing with self
    ##for x in vertexSharedFacesCount[currentVertex-1]:
        ##if x > 1:
            ##print(x, currentVertex)
            


#print(vertexSharedFacesCount)
#print(vertexDictOfSharedEdges)

tempVertexDictOfSharedEdges = copy.deepcopy(vertexDictOfSharedEdges)
print(tempVertexDictOfSharedEdges)

#Once this removes an element from the dictionary, it doesn't check the next value. use temp dictionary
for currentVertex in vertexDictOfSharedEdges:
    for connectedVertex in vertexDictOfSharedEdges[currentVertex]:
        #print(currentVertex, connectedVertex)
        if connectedVertex == currentVertex:
            tempVertexDictOfSharedEdges[currentVertex].remove(connectedVertex)
            #print("removed", connectedVertex)
            
#print(vertexDictOfSharedEdges)

vertexDictOfSharedEdges = copy.deepcopy(tempVertexDictOfSharedEdges)
    
print(vertexDictOfSharedEdges)  



{1: [29, 43, 25, 26, 36, 37, 33, 40, 49, 50, 30, 44], 2: [29, 62, 25, 27, 36, 39, 34, 53, 56, 58, 32, 64], 3: [43, 73, 40, 41, 49, 51, 47, 66, 68, 70, 46, 74], 4: [43, 62, 36, 38, 33, 41, 68, 69, 34, 60, 45, 63], 5: [29, 73, 26, 27, 49, 52, 47, 54, 56, 57, 31, 75], 6: [62, 73, 56, 59, 53, 60, 68, 71, 54, 66, 65, 76], 7: [86, 97, 80, 83, 77, 84, 92, 95, 78, 90, 89, 100], 8: [44, 86, 37, 38, 33, 42, 92, 93, 35, 84, 45, 87], 9: [30, 97, 26, 28, 80, 81, 78, 48, 50, 52, 31, 99], 10: [44, 97, 40, 42, 92, 94, 90, 48, 50, 51, 46, 98], 11: [30, 86, 25, 28, 37, 39, 35, 77, 80, 82, 32, 88], 12: [87, 113, 84, 85, 108, 111, 107, 91, 93, 95, 89, 116], 13: [45, 113, 41, 42, 69, 70, 67, 91, 93, 94, 46, 114], 14: [63, 87, 38, 39, 34, 61, 108, 109, 35, 85, 64, 88], 15: [63, 113, 60, 61, 108, 110, 107, 67, 69, 71, 65, 115], 16: [98, 114, 94, 95, 90, 96, 117, 120, 91, 112, 100, 116], 17: [74, 98, 51, 52, 47, 72, 117, 118, 48, 96, 75, 99], 18: [74, 114, 70, 71, 66, 72, 117, 119, 67, 112, 76, 115], 19: [99,

# Fill out matrix of vertex to vertex distances

In [37]:
##Implement Djikstra's with min heap

rows, cols = (maxVert, maxVert)
vertexToVertexDistance = [[999 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)

for i in range(0,maxVert):
    vertexToVertexDistance[i][i] = 0

#print( vertexToVertexDistance)

edgesAway = 1

while np.max(vertexToVertexDistance) == 999:
    for vertex in vertexDictOfSharedEdges:
        for secondVertex in vertexDictOfSharedEdges:
            if vertexToVertexDistance[vertex-1][secondVertex-1] == edgesAway-1:
                #print(secondVertex, " is ", edgesAway, " away")
                for vertXAway in vertexDictOfSharedEdges[secondVertex]:
                    if vertexToVertexDistance[vertex-1][vertXAway-1] == 999:
                        vertexToVertexDistance[vertex-1][vertXAway-1] = edgesAway
    edgesAway += 1
#print(vertexToVertexDistance)

#print(np.amax(vertexToVertexDistance))

#This just confirms that each vertex has a max distance away of 5, and only a single vertex at that distance
for i in range(0,120):
    print(i, max(vertexToVertexDistance[i]), vertexToVertexDistance[i].count(max(vertexToVertexDistance[i])))

0 5 1
1 5 1
2 5 1
3 5 1
4 5 1
5 5 1
6 5 1
7 5 1
8 5 1
9 5 1
10 5 1
11 5 1
12 5 1
13 5 1
14 5 1
15 5 1
16 5 1
17 5 1
18 5 1
19 5 1
20 5 1
21 5 1
22 5 1
23 5 1
24 5 1
25 5 1
26 5 1
27 5 1
28 5 1
29 5 1
30 5 1
31 5 1
32 5 1
33 5 1
34 5 1
35 5 1
36 5 1
37 5 1
38 5 1
39 5 1
40 5 1
41 5 1
42 5 1
43 5 1
44 5 1
45 5 1
46 5 1
47 5 1
48 5 1
49 5 1
50 5 1
51 5 1
52 5 1
53 5 1
54 5 1
55 5 1
56 5 1
57 5 1
58 5 1
59 5 1
60 5 1
61 5 1
62 5 1
63 5 1
64 5 1
65 5 1
66 5 1
67 5 1
68 5 1
69 5 1
70 5 1
71 5 1
72 5 1
73 5 1
74 5 1
75 5 1
76 5 1
77 5 1
78 5 1
79 5 1
80 5 1
81 5 1
82 5 1
83 5 1
84 5 1
85 5 1
86 5 1
87 5 1
88 5 1
89 5 1
90 5 1
91 5 1
92 5 1
93 5 1
94 5 1
95 5 1
96 5 1
97 5 1
98 5 1
99 5 1
100 5 1
101 5 1
102 5 1
103 5 1
104 5 1
105 5 1
106 5 1
107 5 1
108 5 1
109 5 1
110 5 1
111 5 1
112 5 1
113 5 1
114 5 1
115 5 1
116 5 1
117 5 1
118 5 1
119 5 1


# Filling out the distance matrix with a min heap Dijkstra's

# Calculate the distance from each maxUntouchedVertex to a covered face

In [9]:
## Create function for running calucations on each vertex in an input list
## Simply returns which of the input vertices is closest

## Premake the matrix of distances

## When called, search through covered faces
## Pull out every vertex in the covered faces sets

## For each maxUntouchedVertex, find distance each vertex in covered faces sets, take min
## Store min as distance from covered faces

## Find maxUntouchedVertex with smallest distance from covered faces

## Return that maxUntouchedVertex

def findNextVertex(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
    dictOfMaxedVertDistanceToFace = {}            
    for vertex in listOfMaxedFaceVerts:
        dictOfMaxedVertDistanceToFace[vertex] = []
        closestDistance = 999999
        for vertOnFace in verticesTouchingCoveredFaces:
            if vertexToVertexDistance[vertex-1][vertOnFace -1] < closestDistance:
                closestDistance = vertexToVertexDistance[vertex-1][vertOnFace -1]
        #print(vertex, closestDistance)
        dictOfMaxedVertDistanceToFace[vertex] = closestDistance
            
        
    #print(dictOfMaxedVertDistanceToFace)

    ## Find maxUntouchedVertex with smallest distance from covered faces
    closestVert = 9999
    closestVertDist = 9999
    
    for vertex in dictOfMaxedVertDistanceToFace:
        if dictOfMaxedVertDistanceToFace[vertex] < closestVertDist:
            closestVert = vertex
            closestVertDist = dictOfMaxedVertDistanceToFace[vertex]
            #print("New closest is ", vertex)
    return(closestVert)

# New Attempt At TieBreaker

In [10]:
## For each maxed vertex, find distance to each covered faces
## Can refernce to vertexToVertex for distances I think
## Find minimum dist to touched face, and count how mnay faces are that distance
## Can have 2d array that holds info
    ## 3 Columns, vertex, minDist, and countOfMinDist

def findNextVertexFixed(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
                
    ##for each maxUntouched, find distance to each verticesTouchingCoveredFaces
    dictOfMaxedVertDistanceToCoveredFaces = {}            
    for vertex in listOfMaxedFaceVerts:
        dictOfMaxedVertDistanceToCoveredFaces[vertex] = []
        for vertOnFace in verticesTouchingCoveredFaces:
            dictOfMaxedVertDistanceToCoveredFaces[vertex] = np.append(dictOfMaxedVertDistanceToCoveredFaces[vertex], vertexToVertexDistance[vertex-1][vertOnFace -1])
            dictOfMaxedVertDistanceToCoveredFaces[vertex] = dictOfMaxedVertDistanceToCoveredFaces[vertex].astype(int)
    #print("")
    print("dictOfMaxedVertDistanceToCoveredFaces is now:")
    print(dictOfMaxedVertDistanceToCoveredFaces)
    #print("")
    
    ##Find total minimum distance to a covered face
    minDist = 999999
    for x in dictOfMaxedVertDistanceToCoveredFaces:
        if min(dictOfMaxedVertDistanceToCoveredFaces[x]) < minDist:
            minDist = min(dictOfMaxedVertDistanceToCoveredFaces[x])

    
    
    ##For each maxUntouched find how many faces are the minimum distances away
    ##Store # of faces closest to maxedVerts
    dictOfNumberClosestFaces = {}
    for vertex in dictOfMaxedVertDistanceToCoveredFaces:
        dictOfNumberClosestFaces[vertex] = []
        count = (dictOfMaxedVertDistanceToCoveredFaces[vertex] == minDist).sum()
        dictOfNumberClosestFaces[vertex] = count
    maxedClosestFaces = max(dictOfNumberClosestFaces.values())
        
        ## Stores all maxed
    vertsToChooseFrom = []
    for vertex in dictOfNumberClosestFaces:
        if dictOfNumberClosestFaces.get(vertex) == maxedClosestFaces:
            vertsToChooseFrom = np.append(vertsToChooseFrom, vertex)
    ##Pick one randomly / take smallest or largest
    closestVert = random.choice(vertsToChooseFrom)
        
    return(closestVert)

# Randomly select starting vertex

In [11]:
## Select Random Starting vertex
startingVertex = random.randint(1,maxVert+1)
lastVisitedVertex = copy.deepcopy(startingVertex)
print(startingVertex)

54


# Add current vertex to solution set

In [12]:
## Add vertex to a solution attempt
solutionVertices = list()
solutionVertices.append(lastVisitedVertex)
print(solutionVertices)

[54]


In [13]:
vertexDictOfTouchedFaces[lastVisitedVertex]

[array([ 5, 47, 54, 75]),
 array([ 5, 47, 54, 73]),
 array([ 5, 54, 56, 57]),
 array([ 5, 54, 56, 73]),
 array([ 5, 54, 57, 75]),
 array([ 6, 54, 56, 59]),
 array([ 6, 54, 66, 76]),
 array([ 6, 54, 66, 73]),
 array([ 6, 54, 56, 73]),
 array([ 6, 54, 59, 76]),
 array([21, 54, 57, 59]),
 array([21, 54, 72, 76]),
 array([21, 54, 72, 75]),
 array([21, 54, 57, 75]),
 array([21, 54, 59, 76]),
 array([54, 56, 57, 59]),
 array([47, 54, 72, 75]),
 array([47, 54, 66, 73]),
 array([54, 66, 72, 76]),
 array([47, 54, 66, 72])]

# Processes Faces Touched by current vertex
# Counts faces as covered for master array
# Removes face from each vertex's untouched faces dictionary

In [14]:
## Find touched faces
currentTouchedFaces = vertexDictOfTouchedFaces[lastVisitedVertex]
#print(currentTouchedFaces)


## Count all as "covered"
    ## Add to a set of "covered faces"
##Needed to set size of array, make sure to remove later!!!!!!!!!!!!!!!!!!!!!!!!
##facesToBeAdded = array([[-1,-1,-1,-1]])
facesToBeAdded = np.ndarray(shape = (1, len(inputMatrix[0])))
facesToBeAdded.fill(-1)
facesToBeAdded = facesToBeAdded.astype(int)

for x in currentTouchedFaces:
    facesToBeAdded = np.vstack([facesToBeAdded, x]) ## DUMB

#print(facesToBeAdded)

masterCoveredFaces = np.vstack([masterCoveredFaces, facesToBeAdded])

##Removes the size setting rows
removed = (masterCoveredFaces == -1).sum(1)
fixed = masterCoveredFaces[removed == 0, :]

   
masterCoveredFaces = copy.deepcopy(fixed)

#print(masterCoveredFaces)



## Remove from the current vertex's list of uncovered faces
for x in vertexDictOfTouchingUncoveredFaces:
    ##make temp
    ##tempXTouchingUncovered = array([[-1,-1,-1,-1]])
    ##print(tempXTouchingUncovered)
    
    tempXTouchingUncovered = np.ndarray(shape = (1, len(inputMatrix[0])))
    tempXTouchingUncovered.fill(-1)
    tempXTouchingUncovered = tempXTouchingUncovered.astype(int)
    
    
    for y in vertexDictOfTouchingUncoveredFaces[x]:
        ## make inZ
        inZ = False
        for z in currentTouchedFaces:
            if np.array_equal(y,z):
                #Set inZ to true
                inZ = True
                #print("These are equal")
        ## if not inZ, add to temp
        if not inZ:
            tempXTouchingUncovered = np.vstack([tempXTouchingUncovered, y])
    ## fix temp
        #print(tempXTouchingUncovered)
        #print("Me before fixing: ", tempXTouchingUncovered)
        removed = (tempXTouchingUncovered == -1).sum(1)
        fixed = tempXTouchingUncovered[removed == 0, :]
        #print(x, fixed)
        tempXTouchingUncovered = copy.deepcopy(fixed)
        
    
    removed = (tempXTouchingUncovered == -1).sum(1)
    fixed = tempXTouchingUncovered[removed == 0, :]
    #print(x, fixed)
    tempXTouchingUncovered = copy.deepcopy(fixed)
        
    ## make temp x's uncovered touching
    #print(x,tempXTouchingUncovered)
    vertexDictOfTouchingUncoveredFaces[x] = copy.deepcopy(tempXTouchingUncovered)
    #print("this",vertexDictOfTouchingUncoveredFaces[x])
                                       
        
        
print(vertexDictOfTouchingUncoveredFaces)

{1: array([[ 1, 29, 43, 49],
       [ 1, 29, 36, 43],
       [ 1, 25, 26, 30],
       [ 1, 25, 26, 29],
       [ 1, 25, 36, 37],
       [ 1, 33, 36, 37],
       [ 1, 33, 40, 44],
       [ 1, 33, 40, 43],
       [ 1, 40, 49, 50],
       [ 1, 26, 49, 50],
       [ 1, 30, 44, 50],
       [ 1, 30, 37, 44],
       [ 1, 25, 29, 36],
       [ 1, 33, 36, 43],
       [ 1, 40, 43, 49],
       [ 1, 26, 29, 49],
       [ 1, 25, 30, 37],
       [ 1, 33, 37, 44],
       [ 1, 40, 44, 50],
       [ 1, 26, 30, 50]]), 2: array([[ 2, 29, 56, 62],
       [ 2, 29, 36, 62],
       [ 2, 25, 27, 32],
       [ 2, 25, 27, 29],
       [ 2, 25, 36, 39],
       [ 2, 34, 36, 39],
       [ 2, 34, 53, 64],
       [ 2, 34, 53, 62],
       [ 2, 53, 56, 58],
       [ 2, 27, 56, 58],
       [ 2, 32, 58, 64],
       [ 2, 32, 39, 64],
       [ 2, 25, 29, 36],
       [ 2, 34, 36, 62],
       [ 2, 53, 56, 62],
       [ 2, 27, 29, 56],
       [ 2, 25, 32, 39],
       [ 2, 34, 39, 64],
       [ 2, 53, 58, 64],
       [ 2, 27, 

# Removes faces touched by current vertex from master uncovered array

In [15]:
## Remove touched faces from "uncovered" list of faces
## Removing is hard when not sure which row location removing from.
## Maybe try just rewriting without the covered ones?

#tempUncoveredFaces = array([[-1,-1,-1,-1]])
tempUncoveredFaces = np.ndarray(shape = (1, len(inputMatrix[0])))
tempUncoveredFaces.fill(-1)
tempUncoveredFaces = tempUncoveredFaces.astype(int)
print(vertexDictOfTouchingUncoveredFaces)

for x in masterUncoveredFaces:
    inY = False
    for y in vertexDictOfTouchedFaces[lastVisitedVertex]:
        #print(vertexDictOfTouchedFaces[lastVisitedVertex])
        if np.array_equal(x,y):
            inY = True
    if not inY:
        tempUncoveredFaces = np.vstack([tempUncoveredFaces,x])
#print(tempUncoveredFaces)

removed = (tempUncoveredFaces == -1).sum(1)
fixed = tempUncoveredFaces[removed == 0, :]

masterUncoveredFaces = copy.deepcopy(fixed)

print("Faces Not Yet Covered") 
print(masterUncoveredFaces)
print("")
print("Faces covered") 
print(masterCoveredFaces)


{1: array([[ 1, 29, 43, 49],
       [ 1, 29, 36, 43],
       [ 1, 25, 26, 30],
       [ 1, 25, 26, 29],
       [ 1, 25, 36, 37],
       [ 1, 33, 36, 37],
       [ 1, 33, 40, 44],
       [ 1, 33, 40, 43],
       [ 1, 40, 49, 50],
       [ 1, 26, 49, 50],
       [ 1, 30, 44, 50],
       [ 1, 30, 37, 44],
       [ 1, 25, 29, 36],
       [ 1, 33, 36, 43],
       [ 1, 40, 43, 49],
       [ 1, 26, 29, 49],
       [ 1, 25, 30, 37],
       [ 1, 33, 37, 44],
       [ 1, 40, 44, 50],
       [ 1, 26, 30, 50]]), 2: array([[ 2, 29, 56, 62],
       [ 2, 29, 36, 62],
       [ 2, 25, 27, 32],
       [ 2, 25, 27, 29],
       [ 2, 25, 36, 39],
       [ 2, 34, 36, 39],
       [ 2, 34, 53, 64],
       [ 2, 34, 53, 62],
       [ 2, 53, 56, 58],
       [ 2, 27, 56, 58],
       [ 2, 32, 58, 64],
       [ 2, 32, 39, 64],
       [ 2, 25, 29, 36],
       [ 2, 34, 36, 62],
       [ 2, 53, 56, 62],
       [ 2, 27, 29, 56],
       [ 2, 25, 32, 39],
       [ 2, 34, 39, 64],
       [ 2, 53, 58, 64],
       [ 2, 27, 

# Find all vertices with highest uncovered faces

In [16]:
## Finding Next Vertex
    ## Find all vertices that touch maximum number of uncovered faces
maxUntouched = 0
maxUntouchedVerts = []
for x in vertexDictOfTouchingUncoveredFaces:
    if len(vertexDictOfTouchingUncoveredFaces[x]) > maxUntouched:
        maxUntouched = len(vertexDictOfTouchingUncoveredFaces[x])
    #print(x, len(vertexDictOfTouchingUncoveredFaces[x]))
    #print("max ", maxUntouched )
    
for x in vertexDictOfTouchingUncoveredFaces:
    if len(vertexDictOfTouchingUncoveredFaces[x]) == maxUntouched:
            maxUntouchedVerts.append(x)
print(maxUntouchedVerts)
print(maxUntouchedVerts[0])
    
    
    ## Calculate which of vertices in previous step are closest to a covered face 
if len(maxUntouchedVerts) == 1:
    lastVisitedVertex = maxUntouchedVerts[0]
elif len(maxUntouchedVerts) == 0:
    print("SOMETHING IS VERY WRONG HERE, NO MAXIMIZED VERTS")
else:
    print("we are calculating")
    lastVisitedVertex = findNextVertexFixed(maxUntouchedVerts)
    
    
print("The next vertex is" , lastVisitedVertex)

[1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 55, 58, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
1
we are calculating
dictOfMaxedVertDistanceToCoveredFaces is now:
{1: array([2, 2, 3, 3, 2, 2, 3, 3, 3, 3, 4, 4, 3]), 2: array([2, 3, 2, 3, 2, 1, 2, 2, 2, 3, 3, 3, 3]), 3: array([2, 1, 2, 2, 1, 2, 3, 2, 3, 1, 2, 3, 2]), 4: array([3, 3, 3, 4, 2, 2, 3, 2, 3, 2, 3, 4, 3]), 7: array([4, 4, 4, 3, 4, 4, 4, 5, 4, 4, 4, 4, 4]), 8: array([4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 5, 4]), 9: array([2, 2, 3, 2, 3, 3, 2, 4, 3, 3, 4, 3, 3]), 10: array([3, 2, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 3]), 11: array([3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4]), 12: array([5, 4, 4, 4, 4, 4,

# Check if all faces covered, if yes, print out solution, covered and uncovered

In [17]:
## Check if all faces covered
print(np.array_equal(inputMatrix, masterCoveredFaces))

## If yes, output list of vertices as solution
## Covered face must be 1 version of all, and uncovered must be empty
if np.array_equal(inputMatrix, masterCoveredFaces) and len(masterUncoveredFaces) == 0:
    print("Solution Set")
    print(solutionVertices)
    print(len(solutionVertices), " vertices in solution set")
    print( " ")
    print("Uncoverd faces")
    print(masterUncoveredFaces)
    print( " ")
    print("Covered Faces")
    print(masterCoveredFaces)

else:
    lastVisitedVertex = findNextVertexFixed(maxUntouchedVerts)


    
print(lastVisitedVertex.astype(int))
    ## If no, select next Vertex

False
dictOfMaxedVertDistanceToCoveredFaces is now:
{1: array([2, 2, 3, 3, 2, 2, 3, 3, 3, 3, 4, 4, 3]), 2: array([2, 3, 2, 3, 2, 1, 2, 2, 2, 3, 3, 3, 3]), 3: array([2, 1, 2, 2, 1, 2, 3, 2, 3, 1, 2, 3, 2]), 4: array([3, 3, 3, 4, 2, 2, 3, 2, 3, 2, 3, 4, 3]), 7: array([4, 4, 4, 3, 4, 4, 4, 5, 4, 4, 4, 4, 4]), 8: array([4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 5, 4]), 9: array([2, 2, 3, 2, 3, 3, 2, 4, 3, 3, 4, 3, 3]), 10: array([3, 2, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 3]), 11: array([3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4]), 12: array([5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4]), 13: array([4, 3, 3, 4, 3, 4, 4, 3, 4, 2, 3, 4, 3]), 14: array([4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 3, 4, 4]), 15: array([4, 3, 3, 4, 3, 3, 3, 2, 2, 2, 2, 3, 3]), 16: array([4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 2]), 17: array([2, 1, 2, 1, 2, 3, 2, 3, 3, 2, 2, 2, 1]), 18: array([3, 2, 2, 2, 2, 3, 3, 2, 2, 1, 1, 2, 1]), 19: array([3, 3, 3, 2, 4, 3, 2, 4, 3, 3, 3, 2, 2]), 20: array([2, 3, 2, 2, 3, 2, 1, 3, 2, 3, 3, 2, 3]), 22: array([4, 4, 4